# Reshaping Data

## About the data
In this notebook, we will using daily temperature data from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). We will use the Global Historical Climatology Network - Daily (GHCND) dataset; see the documentation [here](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

This data was collected for New York City for October 2018, using the Boonton 1 station (GHCND:USC00280907). It contains:
- the daily minimum temperature (`TMIN`)
- the daily maximum temperature (`TMAX`)
- the daily temperature at time of observation (`TOBS`)

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Setup
We need to import `pandas` and read in the long format data to get started:

In [11]:
pd.__version__

'2.0.3'

In [1]:
import pandas as pd

long_df = pd.read_csv(
    '../data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

,datatype,date,temp_C,temp_F
0,TMAX,2018-10-01,21.1,69.98
1,TMIN,2018-10-01,8.9,48.02
2,TOBS,2018-10-01,13.9,57.02
3,TMAX,2018-10-02,23.9,75.02
4,TMIN,2018-10-02,13.9,57.02


## Transposing
Transposing swaps the rows and the columns. We use the `T` attribute to do so:

In [2]:
long_df.set_index('date').head(6).T

date,2018-10-01,2018-10-01,2018-10-01,2018-10-02,2018-10-02,2018-10-02
datatype,TMAX,TMIN,TOBS,TMAX,TMIN,TOBS
temp_C,21.1,8.9,13.9,23.9,13.9,17.2
temp_F,69.98,48.02,57.02,75.02,57.02,62.96


## Pivoting
Going from long to wide format.

### `pivot()`
We can restructure our data by picking a column to go in the index (`index`), a column whose unique values will become column names (`columns`), and the values to place in those columns (`values`). The `pivot()` method can be used when we don't need to perform any aggregation in addition to our restructuring (when our index is unique); if this is not the case, we need the `pivot_table()` method. 

In [3]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

datatype,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


Now that the data is pivoted, we have wide format data that we can grab summary statistics with:

In [4]:
pivoted_df.describe()

datatype,TMAX,TMIN,TOBS
count,31.000000,31.000000,31.000000
mean,16.829032,7.561290,10.022581
std,5.714962,6.513252,6.596550
min,7.800000,-1.100000,-1.100000
25%,12.750000,2.500000,5.550000
50%,16.100000,6.700000,8.300000
75%,21.950000,13.600000,16.100000
max,26.700000,17.800000,21.700000


We can also provide multiple values to pivot on, which will result in a hierarchical index:

In [5]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

With the hierarchical index, if we want to select `TMIN` in Fahrenheit, we will first need to select `temp_F` and then `TMIN`:

In [6]:
pivoted_df['temp_F']['TMIN'].head()

date
2018-10-01    48.02
2018-10-02    57.02
2018-10-03    60.08
2018-10-04    53.06
2018-10-05    53.06
Name: TMIN, dtype: float64

### `unstack()`

We have been working with a single index so far; however, we can create an index from any number of columns with `set_index()`. This gives us an index of type `MultiIndex`, where the outermost level corresponds to the first element in the list provided to `set_index()`:

In [7]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', 'datatype'])

Notice there are now 2 index sections of the dataframe:

In [8]:
multi_index_df.head()

temp_C  temp_F
date       datatype                
2018-10-01 TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02

With an index of type `MultiIndex`, we can no longer use `pivot()`. We must now use `unstack()`, which by default moves the innermost index onto the columns:

In [9]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

The `unstack()` method also provides the `fill_value` parameter, which let's us fill-in any `NaN` values that might arise from this restructuring of the data. Consider the case that we have data for the average temperature on October 1, 2018, but no other date:

In [129]:
# additional_row = pd.DataFrame({
#     'date': ['2018-10-01', '2018-10-01'],
#     'datatype': ['TAVG'], 
#     'temp_C': [10], 
#     'temp_F': [50]
# },index=(['date', 'datatype']))

# 

# additional_row.set_index(['date','datatype']).sort_index()

index = pd.MultiIndex.from_tuples([("2018-10-01", "TAVG")], names=["date", "datatype"])
data = {"temp_C": [10], "temp_F": [50]}

additional_row = pd.DataFrame(data, index=index)


additional_row


# long_df[['date']] = long_df[['date']].astype('datetime64[ns]')


# mi = pd.MultiIndex.from_product([['2018-10-01'],['TAVG'],[10], [50]
#  ],names=['date', 'datatype', 'temp_C', 'temp_F'])

# mi.to_frame()

# additional_row

extra_data = pd.concat([long_df, additional_row],ignore_index=False)
extra_data.set_index(['date','datatype']).sort_index()

extra_data




,datatype,date,temp_C,temp_F
0,TMAX,2018-10-01,21.1,69.98
1,TMIN,2018-10-01,8.9,48.02
2,TOBS,2018-10-01,13.9,57.02
3,TMAX,2018-10-02,23.9,75.02
4,TMIN,2018-10-02,13.9,57.02
...,...,...,...,...
89,TOBS,2018-10-30,5.0,41.00
90,TMAX,2018-10-31,12.2,53.96
91,TMIN,2018-10-31,0.0,32.00
92,TOBS,2018-10-31,0.0,32.00


In [10]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

AttributeError: 'DataFrame' object has no attribute 'append'

If we use `unstack()` in this case, we will have `NaN` for the `TAVG` columns every day but October 1, 2018:

In [12]:
extra_data.unstack().head()

NameError: name 'extra_data' is not defined

To address this, we can pass in an appropriate `fill_value`. However, we are restricted to passing in a value for this, not a strategy (like we saw with `fillna()`), so while `-40` is definitely not be the best value, we can use it to illustrate how this works, since this is the temperature at which Fahrenheit and Celsius are equal:

In [ ]:
extra_data.unstack(fill_value=-40).head()

## Melting
Going from wide to long format.

### Setup

In [ ]:
wide_df = pd.read_csv('../data/wide_data.csv')
wide_df.head()

### `melt()`
In order to go from wide format to long format, we use the `melt()` method. We have to specify:
- `id_vars`: which column(s) uniquely identify a row in the wide format (`date`, here)
- `value_vars`: the column(s) that contain(s) the values (`TMAX`, `TMIN`, and `TOBS`, here)

Optionally, we can also provide:
- `value_name`: what to call the column that will contain all the values once melted
- `var_name`: what to call the column that will contain the names of the variables being measured

In [ ]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

### `stack()`
Another option is `stack()`, which will pivot the columns of the dataframe into the innermost level of the index (resulting in an index of type `MultiIndex`). To illustrate this, let's set our index to be the `date` column:

In [ ]:
wide_df.set_index('date', inplace=True)
wide_df.head()

By running `stack()` now, we will create a second level in our index which will contain the column names of our dataframe (`TMAX`, `TMIN`, `TOBS`). This will leave us with a `Series` object containing the values:

In [ ]:
stacked_series = wide_df.stack()
stacked_series.head()

We can use the `to_frame()` method on our `Series` object to turn it into a `DataFrame` object. Since the series doesn't have a name at the moment, we will pass in the name as an argument:

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

Once again, we have an index of type `MultiIndex`:

In [ ]:
stacked_df.head().index

Unfortunately, we don't have a name for the `datatype` level:

In [ ]:
stacked_df.index.names

We can use `set_names()` to address this though:

In [ ]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names